## Lesson 2: Sequential Chats and Customer Onboarding

https://learn.deeplearning.ai/courses/ai-agentic-design-patterns-with-autogen/lesson/3/sequential-chats-and-customer-onboarding

In [1]:
from autogen import ConversableAgent
from dotenv import find_dotenv, load_dotenv

In [12]:
load_dotenv(find_dotenv())

MODEL_NAME = "glm-4-air"
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/"

llm_config = {"model": MODEL_NAME, "base_url": BASE_URL, "temperature": 0.5}

### Creating the needed agents

In [13]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message="""You are a helpful customer onboarding agent,
        you are here to help new customers get started with our product.
        Your job is to gather customer's name and location.
        Do not ask for other information. Return 'TERMINATE'
        when you have gathered all the information.
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic Preference Agent",
    system_message="""You are a helpful customer onboarding agent,
        you are here to help new customers get started with our product.
        Your job is to gather customer's preferences on news topics.
        Do not ask for other information. Return 'TERMINATE'
        when you have gathered all the information.
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message="""You are a helpful customer service agent
        here to provide fun for the customer based on the user's
        personal information and topic preferences.
        This could include fun facts, jokes, or interesting stories.
        Make sure to make it engaging and fun!
        Return 'TERMINATE' when you are done.
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

### Creating tasks
Now you can craft a series of tasks to facilitate the onboarding process.

In [14]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": """
            Hello, I'm here to help you get started with our product.
            Could you tell me your name and location?
        """,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": """
                Return the customer information into as
                JSON object only: {'name': '', 'location': ''}
            """,
        },
        "max_turns": 2,
        "clear_hisotry": True,
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": """
            Great! Could you tell me what topics you are interested in reading about?
        """,
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_hisotry": False,
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find somthing fun to read.",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
    },
]

### Start the onboarding process

In [5]:
from autogen import initiate_chats

In [15]:
chat_results = initiate_chats(chat_queue=chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):


            Hello, I'm here to help you get started with our product.
            Could you tell me your name and location?
        

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

I'm Bob and I'm in London right now.

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you, Bob. I've noted that you're in London. Is there anything else I can assist you with before we proceed? If not, I will mark this as complete and move forward with the onboarding process. TERMINATE

-----------------------------------------------------------------------

### Print out the summary

In [16]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

, that's all I needed to get started. You can proceed with the onboarding process for me.


, artificial intelligence, space exploration, technological advancements, philosophy of mind, cognitive science, quantum computing, historical fiction, cybersecurity, and the impact of technology on society. However, my interests are not limited to these topics as I am designed to assist with a wide range of subjects based on user queries and needs. If you have a specific topic in mind, feel free to let me know, and I can provide information or discuss it with you.


the current thread if you'd like to move on to something else, or CONTINUE exploring the current theme.




### Print out the cost

In [17]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0, 'glm-4-air': {'cost': 0, 'prompt_tokens': 203, 'completion_tokens': 71, 'total_tokens': 274}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'glm-4-air': {'cost': 0, 'prompt_tokens': 203, 'completion_tokens': 71, 'total_tokens': 274}}}


{'usage_including_cached_inference': {'total_cost': 0, 'glm-4-air': {'cost': 0, 'prompt_tokens': 49, 'completion_tokens': 90, 'total_tokens': 139}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'glm-4-air': {'cost': 0, 'prompt_tokens': 49, 'completion_tokens': 90, 'total_tokens': 139}}}


{'usage_including_cached_inference': {'total_cost': 0, 'glm-4-air': {'cost': 0, 'prompt_tokens': 752, 'completion_tokens': 455, 'total_tokens': 1207}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'glm-4-air': {'cost': 0, 'prompt_tokens': 752, 'completion_tokens': 455, 'total_tokens': 1207}}}


